In [6]:
import os.path

from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain_community.vectorstores import Chroma
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import Language
from langchain_community.document_loaders.parsers import LanguageParser
from langchain_community.document_loaders.generic import GenericLoader

repository_root = "/Users/raynor/workspace/xxx/"

fitpetmall_backend_documents = []

for path in ["skeds", ]:
    loader = GenericLoader.from_filesystem(
        os.path.join(repository_root, path),
        glob="**/*",
        suffixes=[".py"],
        parser=LanguageParser(language=Language.PYTHON.value, parser_threshold=30),
    )
    fitpetmall_backend_documents.extend(loader.load())

embeddings = OllamaEmbeddings(model="mxbai-embed-large")
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings,
    LocalFileStore("./.store/"),
    namespace=embeddings.model,
)

db = Chroma.from_documents(
    fitpetmall_backend_documents,
    cached_embeddings,
    persist_directory="./.vector/",
)


<unknown>:62: SyntaxWarning: invalid escape sequence '\ '
<unknown>:62: SyntaxWarning: invalid escape sequence '\ '
<unknown>:62: SyntaxWarning: invalid escape sequence '\ '
<unknown>:102: SyntaxWarning: invalid escape sequence '\d'
<unknown>:102: SyntaxWarning: invalid escape sequence '\d'
<unknown>:102: SyntaxWarning: invalid escape sequence '\d'


In [12]:
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate

retriever = db.as_retriever(search_type="mmr", search_kwargs={"k": 10})
bm25_retriever = BM25Retriever.from_documents(fitpetmall_backend_documents)
bm25_retriever.k = 10

ensemble_retriever = EnsembleRetriever(
    retrievers=[retriever, bm25_retriever],
    weights=[0.4, 0.6],
    search_type="mmr",
)

In [17]:
prompt = PromptTemplate.from_template("""
당신은 핏펫몰 백엔드 개발자 입니다. 당신의 임무는 주어진 질문에 대해여 최대한 주어진 코드를 활용하여 답변하는 것입니다.
최대한 자세하게 답변하고, 주어진 코드로 답을 찾을수 없는 경우 `정보 부족으로 답변이 불가능합니다`로 답변 해주세요. 
답변은 출처(source) 코드를 반드시 표기해주세요.

# 참고코드:
{context}

# 질문:
{question}

출처:
- source1
- source2
- ...
""")

llm = ChatOllama(model="llama3.1", temparature=0)
chain = {"context": ensemble_retriever, "question": RunnablePassthrough()} | prompt | llm | StrOutputParser()

for token in chain.stream({"context": "", "question": "주문 취소 로직에 대해서 알려줘"}):
    print(token, end="", flush=True)

ResponseError: invalid input type